# 构建高级有效的RAG

本次我们探索下，如何使用LlamaIndex构建一个有效且高效的RAG应用。

首先，我们来回顾下，什么是RAG?
> RAG（Retrieval-Augmented Generation，检索增强生成）是一种结合信息检索与自然语言生成的技术，旨在提升大型语言模型（LLM）在特定任务中的准确性和实用性。
其核心思想是：在生成回答时，模型不仅依赖自身的预训练知识，还会动态检索外部知识库中的相关信息，并将这些信息整合到最终输出中。

![Image](../../figures/RAG_define.png)

**RAG的典型流程**
1. 用户提问：例如，“2023年诺贝尔文学奖得主是谁？”。
2. 检索相关文档：从知识库中查找与“2023年诺贝尔文学奖”相关的最新信息。
3. 生成回答：模型结合检索到的信息（如获奖者姓名、背景等）生成答案，例如：“2023年诺贝尔文学奖授予了作家约翰·史密斯，以表彰他在当代文学中的杰出贡献。”

## RAG成功的要求
为了使agent可以回答有用又相关的问题，必须要求RAG系统有2个高级要求：
1. 检索组件必须能够找到与用户查询最相关的文档。
2. 生成组件必须能够有效利用检索到的文档，充分回答用户的查询。

## RAG适用典型的木桶理论场景
![Image](../../figures/RAG_mutong.png)


## 简单的RAG

In [ ]:
! pip install llama_index

> llama_index的使用依赖llm，默认使用open ai的gpt-3.5-turbo模型。因为国内使用openAI受限，所以这里我使用openAi like库使用我自己可访问的远程OPENAI API。这里我选择使用百炼api。

In [ ]:
! pip install llama-index-llms-openai-like

In [ ]:
! pip install llama-index-embeddings-dashscope

In [ ]:
from llama_index.llms.openai_like import OpenAILike
import os

api_key = os.environ.get('DASHSCOPE_API_KEY')

llm = OpenAILike(
    model="qwen-max",
    api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",
    api_key=api_key,
    context_window=128000,
    is_chat_model=True,
    is_function_calling_model=False,
)

response = llm.complete("Hello World!")
print(str(response))

In [ ]:
from llama_index.core import SimpleDirectoryReader,Settings
from llama_index.core.indices.vector_store.base import VectorStoreIndex
from llama_index.embeddings.dashscope import DashScopeEmbedding
# Settings control global defaults
# LlamaIndex默认使用的Embedding模型被替换为百炼的Embedding模型
Settings.embed_model = DashScopeEmbedding(
    model_name="text-embedding-v2"
)
Settings.llm =OpenAILike(
    model="qwen-max",
    api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",
    api_key=api_key,
    context_window=128000,
    is_chat_model=True,
    is_function_calling_model=False,
)
# load data
input_data = ['d:/4px_work/MyNote/AI学习/知识库文档/诺贝尔/诺贝尔化学奖.pdf']
documents = SimpleDirectoryReader(input_files=input_data).load_data()
index = VectorStoreIndex.from_documents(documents=documents)
# # 输出建立好的索引和压缩好的向量示例
# print("输出向量化示例：")
# for i, uuid in enumerate(index.vector_store.data.metadata_dict.keys()):
#     print("文件名：", end='')
#     print(index.vector_store.data.metadata_dict[uuid]['file_name'], end='')
#     print("，文件大小：", index.vector_store.data.metadata_dict[uuid]['file_size'], end='')
#     print("，文件类型：", index.vector_store.data.metadata_dict[uuid]['file_type'])
#     print("压缩后向量：", end='')
#     print(index.vector_store.data.embedding_dict[uuid][:3], '\n')
#     if i > 3:
#         break
query_engine = index.as_query_engine()
result = query_engine.query("请根据文档回答，1950年化学诺贝尔奖得主是谁？")
print(f"ai回答={result}")

*可以看到，询问第一行的诺贝尔得奖可以得到正确答案，但是询问文档偏后的，比如2001年就无法获取结果，但文档中是有的。即使问同一个年份回答结果也很不稳定。*
**这是为什么呢？**

**多次回答结果示例：**
1. ai回答=文档中没有提供1972年诺贝尔化学奖得主的信息。
2. ai回答=1901年的诺贝尔化学奖得主是雅各布斯·亨里克斯·范托夫（Jacobus Henricus van’t Hoff, 1852–1911），他来自荷兰，获奖原因是“发现了化学动力学法则和溶液渗透压”。
3. ai回答=1908年的诺贝尔化学奖得主是欧内斯特·卢瑟福（Ernest Rutherford）。
4. ai回答=文档中没有提供1950年诺贝尔化学奖得主的信息。

## 高效的RAG-优化

> 在RAG高效检索技术中，控制系统效率和性能的一个关键参数是chunk_size，如何确定检索的最佳块大小？这就是 **LlamaIndex Response Evaluation**的用处。
- chunk太小，比如128，关键信息无法获取，尤其是similarity_top_k获取信息时。信息被切割到不同的chunk中，无法提供足够的上下文，导致llm理解不完整。
- chunk太大，比如512，噪音比较大，一次召回很多相似的内容，召回质量下降。且投喂给llm的上下文信息太多响应会更慢。
- 所以，评估最佳的chunk大小，在不牺牲速度的情况下捕获所有必要信息就很关键，一般向量召回，选择256-512之间进行测试，关键词召回就要大一些，比如1000-3000之间，可以保留更多关键词在一个chunk中。

### 使用LlamaIndex Response Evaluation module评估最佳块大小。

In [ ]:
!pip install llama-index pypdf

In [ ]:
!pip install spacy

In [ ]:
import nest_asyncio

nest_asyncio.apply()

from llama_index.llms.openai_like import OpenAILike
import os

api_key = os.environ.get('DASHSCOPE_API_KEY')

from llama_index.core import (
    SimpleDirectoryReader,
    Settings,
    Response,
    VectorStoreIndex,
)
from llama_index.core.evaluation import (
    FaithfulnessEvaluator,
    RelevancyEvaluator,
    EvaluationResult
)
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.dashscope import DashScopeEmbedding

print("done!")


In [ ]:
from llama_index.core.node_parser import (TextSplitter,SentenceSplitter)
from llama_index.core.schema import TextNode
TextNode?

### Faithfulness Evaluator忠诚度评估

In [1]:
import nest_asyncio
nest_asyncio.apply()

from llama_index.llms.openai_like import OpenAILike
import os
from llama_index.core import (
    SimpleDirectoryReader,
    Settings,
    Response,
    VectorStoreIndex,
)
from llama_index.core.evaluation import (
    FaithfulnessEvaluator,
    RelevancyEvaluator,
    EvaluationResult
)
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.dashscope import DashScopeEmbedding
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.readers.file import PDFReader
from llama_index.core.node_parser import (TextSplitter,SimpleNodeParser)
import pandas as pd
import logging
import re
import pdfplumber
from llama_index.core.schema import TextNode, Document

logging.basicConfig(level=logging.INFO)

# 设置API密钥
api_key = os.environ.get('DASHSCOPE_API_KEY')

# 配置全局设置
Settings.embed_model = DashScopeEmbedding(
    model_name="text-embedding-v2"
)
Settings.llm = OpenAILike(
    model="qwen-max",
    api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",
    api_key=api_key,
    context_window=128000,
    is_chat_model=True,
    is_function_calling_model=False,
)


None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


**使用pdfplumber读取器提取带表格的pdf文件，做预处理：**

In [ ]:
pip install pdfplumber

In [23]:
def load_pdf_with_reader(pdf_path):
	"""使用pdfplumber逐页解析PDF，仅解析表格：第一行作为表头，其余作为记录。返回每页一个Document。"""
	documents: list[Document] = []
	try:
		with pdfplumber.open(pdf_path) as pdf:
			print(f"pdfplumber 打开PDF，共 {len(pdf.pages)} 页")
			for page_index, page in enumerate(pdf.pages, start=1):
				tables = page.extract_tables() or []
				
				sections: list[str] = []
				
				# 处理表格：取第一行作为表头，行到记录
				for table_index, table in enumerate(tables, start=1):
                    # 如果 table 是空/None，或者 table 中所有行都是空的（没有一行是非空的），就进入这个 if 分支
					if not table or not any(row for row in table):
						continue
                    # 去除表格每一个值的前后空格，和无效表格数据
					headers = [h.strip() if isinstance(h, str) else "" for h in (table[0] or [])]
					headers = [h for h in headers if h is not None]
					if not headers:
						continue
					records: list[str] = []
					for row in table[1:]:
						if not row:
							continue
						cells = [c.strip() if isinstance(c, str) else "" for c in row]
						kv_pairs = []
						for idx, header in enumerate(headers):
							value = cells[idx].replace("\n","") if idx < len(cells) else ""
							if header:
								kv_pairs.append(f"{header}: {value}")
						if kv_pairs:
							records.append("记录: " + " | ".join(kv_pairs))
					if records:
						sections.append("表头: " + " | ".join(headers))
						sections.extend(records)
				
				# 如果页面有表格，就不再提取页面文本，避免重复
				# 只有在没有表格的情况下才提取页面文本
				if not tables:
					page_text = page.extract_text() or ""
					if page_text.strip():
						sections.append("页面文本: " + page_text.strip())
				
				# 若此页既无表格也无文本，则跳过
				if not sections:
					continue
				
				combined = "\n".join(sections)
				documents.append(
					Document(
						text=combined,
						metadata={
							"page_number": page_index,
							"file_name": os.path.basename(pdf_path),
							"source": "pdfplumber"
						},
					)
				)
		print(f"成功加载PDF文档，共 {len(documents)} 页包含表格的文档")
		return documents
	except Exception as e:
		print(f"pdfplumber 解析失败: {e}")
		# 回退到SimpleDirectoryReader
		return SimpleDirectoryReader(input_files=[pdf_path]).load_data()

In [24]:
    pdf_path = 'd:/4px_work/MyNote/AI学习/知识库文档/诺贝尔/诺贝尔化学奖.pdf'
    
    print("开始加载PDF文档...")
    documents = load_pdf_with_reader(pdf_path)

开始加载PDF文档...
pdfplumber 打开PDF，共 13 页
成功加载PDF文档，共 13 页包含表格的文档


**定义pdf表格文本分割器：**

In [25]:
class PageAwareSentenceSplitter(SentenceSplitter):
	"""分页感知的分割器：
	- 用于已按页聚合的文本；
	- 对含有表头/记录等结构化标记的内容友好；
	- 仅基于表格记录边界与 chunk_size 进行切分。
	"""

	def __init__(self, chunk_size: int = 1024, chunk_overlap: int = 0):
		super().__init__(chunk_size=chunk_size, chunk_overlap=chunk_overlap, separator="\n")

	def split_text(self, text: str) -> list[str]:
		lines = text.split("\n")
		chunks: list[str] = []
		current_chunk: str = ""

		def flush_current():
			nonlocal current_chunk
			if current_chunk.strip():
				chunk_text = current_chunk.rstrip("\n").replace("\n", " ") + "\n"
				chunks.append(chunk_text.strip())
				current_chunk = ""

		for raw_line in lines:
			line = raw_line.rstrip()
			is_boundary = False
			# 仅在记录行作为边界
			if line.startswith("记录:"):
				is_boundary = True

			# 超长也切
			if is_boundary or (len(current_chunk) + len(line) + 1 > self.chunk_size and current_chunk.strip()):
				flush_current()

			current_chunk += (line + "\n")

		flush_current()
		return chunks

**创建优化的索引和查询引擎，使用自定义的分割器**

In [48]:
def create_optimized_index(documents):
    """创建优化的向量索引"""
    splitter = PageAwareSentenceSplitter(chunk_size=512, chunk_overlap=100)

    # 使用自定义分割器切分每页文档并创建节点
    all_nodes: list[TextNode] = []
    for doc in documents:
        chunks = splitter.split_text(doc.text)
        print(f"第{doc.metadata.get('page_number')}页 生成 {len(chunks)} 个chunk")
        for chunk_idx, chunk_text in enumerate(chunks):
            node = TextNode(text=chunk_text, metadata={**doc.metadata, "chunk_index": chunk_idx})
            all_nodes.append(node)

    print(f"总共生成 {len(all_nodes)} 个节点")
    if all_nodes:
        print(f"示例节点预览: {all_nodes[0].text[:200]}...")

    vector_index = VectorStoreIndex(all_nodes, show_progress=True)
    
    return vector_index

def create_optimized_query_engine(vector_index):
    """创建优化的查询引擎"""
    # 创建检索器，增加检索的节点数量
    retriever = VectorIndexRetriever(
        index=vector_index,
        similarity_top_k=3  # 增加检索的top-k数量
    )
    
    # 创建后处理器，过滤低相似度的结果
    postprocessor = SimilarityPostprocessor(similarity_cutoff=0.75)
    
    # 创建查询引擎
    query_engine = RetrieverQueryEngine(
        retriever=retriever,
        node_postprocessors=[postprocessor]
    )
    
    return query_engine

def display_eval_df(response: Response, eval_result: EvaluationResult) -> None:
    """显示评估结果的函数"""
    if response.source_nodes == []:
        print("没有找到相关源文档!")
        return
    
    print(f"找到 {len(response.source_nodes)} 个相关源文档")
    
    # 显示所有源文档
    for i, source_node in enumerate(response.source_nodes):
        print(f"\n源文档 {i+1}:")
        print(f"相似度分数: {getattr(source_node, 'score', 'N/A')}")
        print(f"内容: {source_node.node.text}...")
    
    # 创建评估DataFrame
    eval_df = pd.DataFrame(
        {
            "Response": str(response),
            "Source": response.source_nodes[0].node.text ,
            "Evaluation Result": "Pass" if eval_result.passing else "Fail",
            "Reasoning": eval_result.feedback,
        },
        index=[0],
    )
    
    eval_df = eval_df.style.set_properties(
        **{
            "inline-size": "600px",
            "overflow-wrap": "break-word",
        },
        subset=["Response", "Source"]
    )
    display(eval_df)

In [49]:
def test_query(query_engine, question):
    """测试查询功能"""
    print(f"\n问题: {question}")
    print("-" * 50)
    
    # 执行查询
    response = query_engine.query(question)
    
    print(f"回答: {response}")
    
    # 评估忠诚度
    evaluator = FaithfulnessEvaluator()
    eval_result = evaluator.evaluate_response(response=response)
    
    # 显示评估结果
    display_eval_df(response, eval_result)

In [50]:
print("\n创建向量索引...")
vector_index = create_optimized_index(documents)


创建向量索引...
第1页 生成 3 个chunk
第2页 生成 3 个chunk
第3页 生成 12 个chunk
第4页 生成 17 个chunk
第5页 生成 17 个chunk
第6页 生成 13 个chunk
第7页 生成 14 个chunk
第8页 生成 12 个chunk
第9页 生成 10 个chunk
第10页 生成 10 个chunk
第11页 生成 8 个chunk
第12页 生成 11 个chunk
第13页 生成 4 个chunk
总共生成 134 个节点
示例节点预览: 页面文本: Univ. Chem. 2018, 33 (2), 47−59 47 •知识介绍• doi: 10.3866/PKU.DXHX201710005 www.dxhx.pku.edu.cn 诺贝尔化学奖 王毓明* 华侨大学化学系，福建 泉州 362011 摘要：介绍诺贝尔奖的由来、评选和颁奖仪式，总结了历年诺贝尔化学奖获奖名单。 关键词：诺贝尔奖；化学奖；获奖者 中图分类号：G64；O6 ...


Generating embeddings:   0%|          | 0/134 [00:00<?, ?it/s]

In [39]:
print("\n创建查询引擎...")
query_engine = create_optimized_query_engine(vector_index)


创建查询引擎...


In [57]:
# 测试不同年份的查询
test_questions = [
    "请根据文档查找1930年化学诺贝尔奖获奖记录，回答获奖人，国籍以及获奖原因？年份：1930。请注意：排除未颁奖的年份。",
    # "请根据文档回答，1930年化学诺贝尔奖得主是谁？",
    # "请根据文档回答，1901年化学诺贝尔奖得主是谁？",
    # "请根据文档回答，1910年化学诺贝尔奖得主是谁？",
    # "请根据文档回答，1920年化学诺贝尔奖得主是谁？",
]

print("\n开始测试查询...")
for question in test_questions:
    try:
        response, eval_result = test_query(query_engine, question)
        print("\n" + "="*80 + "\n")
    except Exception as e:
        print(f"查询失败: {e}")
        print("\n" + "="*80 + "\n")


开始测试查询...

问题: 请根据文档查找1930年化学诺贝尔奖获奖记录，回答获奖人，国籍以及获奖原因？年份：1930。请注意：排除未颁奖的年份。
--------------------------------------------------


INFO:httpx:HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"


回答: 在提供的文档信息中，没有找到1930年的诺贝尔化学奖获奖记录。因此，无法提供该年份的获奖人、国籍以及获奖原因的信息。


INFO:openai._base_client:Retrying request to /chat/completions in 0.410019 seconds
INFO:httpx:HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"


找到 10 个相关源文档

源文档 1:
相似度分数: 0.7461395767746684
内容: 记录: 年份: 1933 | 获奖者: 未颁奖 | 国籍:  | 获奖原因:...

源文档 2:
相似度分数: 0.7432820482057909
内容: 记录: 年份: 1940 | 获奖者: 未颁奖 | 国籍:  | 获奖原因:...

源文档 3:
相似度分数: 0.7371382199798351
内容: 记录: 年份: 1917 | 获奖者: 未颁奖 | 国籍:  | 获奖原因:...

源文档 4:
相似度分数: 0.7358993862437786
内容: 记录: 年份: 1924 | 获奖者: 未颁奖 | 国籍:  | 获奖原因:...

源文档 5:
相似度分数: 0.7344615879713348
内容: 记录: 年份: 1919 | 获奖者: 未颁奖 | 国籍:  | 获奖原因:...

源文档 6:
相似度分数: 0.7305975454141291
内容: 记录: 年份: 1916 | 获奖者: 未颁奖 | 国籍:  | 获奖原因:...

源文档 7:
相似度分数: 0.7227237163519016
内容: 记录: 年份: 1942 | 获奖者: 未颁奖 | 国籍:  | 获奖原因:...

源文档 8:
相似度分数: 0.7162990985201991
内容: 记录: 年份: 1941 | 获奖者: 未颁奖 | 国籍:  | 获奖原因:...

源文档 9:
相似度分数: 0.6993902507100683
内容: 表头: 年份 | 获奖者 | 国籍 | 获奖原因...

源文档 10:
相似度分数: 0.6991769875727647
内容: 表头: 年份 | 获奖者 | 国籍 | 获奖原因...


,Response,Source,Evaluation Result,Reasoning
0,在提供的文档信息中，没有找到1930年的诺贝尔化学奖获奖记录。因此，无法提供该年份的获奖人、国籍以及获奖原因的信息。,记录: 年份: 1933 | 获奖者: 未颁奖 | 国籍: | 获奖原因:,Pass,YES 解析：虽然提供的上下文中没有直接提到1930年的诺贝尔化学奖的具体信息，但根据给出的信息模式（即对于某些年份显示“未颁奖”），可以推断出如果1930年确实没有在列表中出现，则可能是因为那一年也没有颁发奖项或者相关信息缺失。因此，说“没有找到1930年的诺贝尔化学奖获奖记录”是与上下文相符合的。但是，需要注意的是，这个答案基于给定的有限信息，并不意味着实际上1930年真的没有颁发诺贝尔化学奖。正确的做法应该是查找更完整的数据来源来验证这一点。不过，按照题目要求，基于给定的上下文，答案为YES。


查询失败: cannot unpack non-iterable NoneType object




In [44]:
def debug_search_specific_year(vector_index, target_year):
    """调试搜索特定年份的信息"""
    print(f"\n调试搜索{target_year}年的信息...")
    
    # 获取所有节点
    all_nodes = vector_index.docstore.docs.values()
    
    # 搜索包含目标年份的节点
    matching_nodes = []
    for node in all_nodes:
        if hasattr(node, 'text') and str(target_year) in node.text:
            matching_nodes.append(node)
    
    print(f"找到 {len(matching_nodes)} 个包含{target_year}年的节点:")
    
    if matching_nodes:
        for i, node in enumerate(matching_nodes):
            print(f"\n节点 {i+1}:")
            print(f"页码: {node.metadata.get('page_number', 'N/A')}")
            print(f"Chunk索引: {node.metadata.get('chunk_index', 'N/A')}")
            print(f"内容: {node.text}")
            print("-" * 50)
    else:
        print(f"没有找到包含{target_year}年的节点！")
        
        # 搜索相近年份
        print(f"\n搜索包含相近年份的节点...")
        for year in range(target_year-2, target_year+3):
            if year != target_year:
                year_nodes = [n for n in all_nodes if hasattr(n, 'text') and str(year) in n.text]
                if year_nodes:
                    print(f"{year}年: 找到 {len(year_nodes)} 个节点")
    
    return matching_nodes

In [45]:
 # 调试搜索1918年的信息
debug_search_specific_year(vector_index, 1918)


调试搜索1918年的信息...
找到 8 个包含1918年的节点:

节点 1:
页码: 4
Chunk索引: 7
相似度分数: N/A
内容: 记录: 年份: 1918 | 获奖者: 弗里茨·哈伯(Fritz Haber, 1868–1934) | 国籍: 德国 | 获奖原因: “对从单质合成氨的研究”“for the synthesis of ammonia from its elements”
--------------------------------------------------

节点 2:
页码: 7
Chunk索引: 3
相似度分数: N/A
内容: 记录: 年份: 1958 | 获奖者: 弗雷德里克·桑格(Frederick Sanger, 1918–2013) | 国籍: 英国 | 获奖原因: “对蛋白质结构组成的研究，特别是对胰岛素的研究”“for his work on the structure of proteins, especially that of insulin”
--------------------------------------------------

节点 3:
页码: 8
Chunk索引: 1
相似度分数: N/A
内容: 记录: 年份: 1969 | 获奖者: 德里克·巴顿(Derek H. R. Barton, 1918–1998)奥德·哈塞尔(Odd Hassel, 1897–1981) | 国籍: 英国挪威 | 获奖原因: “发展了构象的概念及其在化学中的应用”“for their contributions to the development of the concept ofconformation and its application in chemistry”
--------------------------------------------------

节点 4:
页码: 8
Chunk索引: 5
相似度分数: N/A
内容: 记录: 年份: 1973 | 获奖者: 恩斯特·奥托·菲舍尔(Ernst Otto Fischer, 1918–2007)杰弗里·威尔金森(Geoffrey Wilkinson, 1921–1996) | 国籍: 西德英国 | 获奖原因: “对金

[TextNode(id_='540954cd-7a69-46f3-a826-fe706a34eeb6', embedding=None, metadata={'page_number': 4, 'file_name': '诺贝尔化学奖.pdf', 'source': 'pdfplumber', 'chunk_index': 7}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='记录: 年份: 1918 | 获奖者: 弗里茨·哈伯(Fritz Haber, 1868–1934) | 国籍: 德国 | 获奖原因: “对从单质合成氨的研究”“for the synthesis of ammonia from its elements”', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'),
 TextNode(id_='2e753169-176b-42ff-aeb6-59791d77f82c', embedding=None, metadata={'page_number': 7, 'file_name': '诺贝尔化学奖.pdf', 'source': 'pdfplumber', 'chunk_index': 3}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='记录: 年份: 1958 | 获奖者: 弗雷德里克·桑格(Frederick Sanger, 1918–2013) | 国籍: 英国 | 获奖原因: “对蛋白质结构组成的研究，特别是对胰岛素的研究”“f

**接下来，然我们调用FaithfulnessEvaluator的evaluate_response()方法来评估，ai给我们的回答是否正确，只需要传入response即可：**

**然后显示评估结果：**

## 以上参考的文档
* llama_index:
    * [quickstart](https://docs.llamaindex.ai/en/stable/#30-second-quickstart)
    * [openai_like llm](https://docs.llamaindex.ai/en/stable/api_reference/llms/openai_like/)
* RAG:
    * [构建高级 RAG 的指南和技巧](https://baoyu.io/translations/rag/a-cheat-sheet-and-some-recipes-for-building-advanced-rag)
      
* llama_index with 百炼：
    * [llama_index with 百炼](https://help.aliyun.com/zh/model-studio/dashscopellm-in-llamaindex)
      
* [LlamaIndex Chunk Size Optimization Recipe](https://www.llamaindex.ai/blog/evaluating-the-ideal-chunk-size-for-a-rag-system-using-llamaindex-6207e5d3fec5)
* [Evaluating the Ideal Chunk Size for a RAG System using LlamaIndex](https://colab.research.google.com/drive/1LPvJyEON6btMpubYdwySfNs0FuNR9nza?usp=sharing)
* [RagEvaluatorPack](https://llamahub.ai/l/llama-packs/llama-index-packs-rag-evaluator?from=)
  

                  